In [2]:
import os
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import lightning.pytorch as pl
import torch_geometric as tg

torch.seed()

/opt/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <DAC8FDCB-770B-356E-BA9C-E2F40A2AA20E> /opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <AE6DCE26-A528-35ED-BB3D-88890D27E6B9> /opt/anaconda3/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


9832140095817926393

In [69]:
class GCNConv(tg.nn.MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')
        self.linear = nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x --> N, in_channels
        edge_index, _ = tg.utils.add_self_loops(edge_index, num_nodes=x.shape[0])
        # edge_index --> 2, num_edges(E)
        x = self.linear(x)
        messages = self.propagate(edge_index, x=x, size=(x.shape[0], x.shape[0])) # size should be num_nodes, num_nodes
        return messages

    def message(self, x_i, x_j, edge_index, size):
        # x_j --> E, in_channels : feature vectors of the source nodes (nodes from which the edge is directed)
        # x_i --> E, in_channels : feature vectors of the neighbour nodes (nodes towards which the edge is directed)
        row, col = edge_index
        # print("row",row)
        # print("col", col)
        deg = tg.utils.degree(index=row, num_nodes=size[0], dtype=x_j.dtype) # index takes the row from which the edge is directed
        # print("xs")
        # print(x_i)
        # print(x_j)
        # print("deg", deg)
        # print("deg row", deg[row]) # degree of each node of the index col
        # print("deg col", deg[col]) # degree of each node in the nodes towards which the edge goes
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm_factor = deg_inv_sqrt[row] * deg_inv_sqrt[col] # because of undirected graph
        norm_factor = norm_factor.view(-1, 1)
        return norm_factor * x_j
    
    def update(self, aggr_out):
        return aggr_out

In [70]:
class EdgeConv(tg.nn.MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr="max")
        self.mlp = nn.Sequential(
            nn.Linear(2 * in_channels, out_channels),
            nn.LeakyReLU(),
            nn.Linear(out_channels, out_channels),
        )
    
    def forward(self, x, edge_index):
        message = self.propagate(edge_index, x=x, size=(x.shape[0], x.shape[0]))
        return message
    
    def message(self, x_i, x_j):
        diff = x_i - x_j
        msg = torch.cat([x_i, diff], dim=1)
        msg = self.mlp(msg)
        return msg
    
    def update(self, aggr_out):
        return aggr_out

In [ ]:
from torch_geometric.nn import knn_graph 
# GPU accelerated edge convolutions as the operation apparently computes the entire graph sequentially
class KNNConv(EdgeConv):
    def __init__(self, in_channels, out_channels, k=6):
        super(KNNConv, self).__init__(aggr='max')
        self.k = k

    def forward(self, x, batch=None):
        edge_index = knn_graph(x, self.k, batch, loop=False, flow=self.flow) # flow defaults to source_to_target w/reference to message passing
        return super.forward(x, edge_index)

In [101]:
gcn = GCNConv(2, 3)
x = torch.randn(5, 2) # num_nodes * features_per_node
y = torch.randn(5, 3) # num_nodes * features_per_node

# x = torch.arange(0,10).view(5, 2).float()
edge_index = torch.tensor([[0, 1, 1, 2, 3, 4],
                           [1, 4, 2, 1, 3, 3]], dtype=torch.long) # 2, E
adam = optim.Adam(gcn.parameters(), lr=0.001)

# message = gcn.message(x, x, edge_index)
# print(message, message.shape)
x_out = gcn(x, edge_index)
print(x_out, x_out.shape)

tensor([[-0.1076,  0.1549,  0.3280],
        [-0.0326,  0.2252,  1.1151],
        [ 0.0677,  0.1209,  1.0377],
        [ 0.4138,  0.7760,  1.5657],
        [ 0.4418, -0.0877,  1.6486]], grad_fn=<ScatterAddBackward0>) torch.Size([5, 3])


In [102]:
def train(num_steps):
    grads = []
    for _ in range(num_steps):

        adam.zero_grad()
        x_out = gcn(x, edge_index)
        loss = torch.nn.functional.mse_loss(x_out, y)
        loss.backward()
        grads.append(gcn.linear.weight.grad)
        adam.step()
        if _ % 100 == 0:
            print(loss.item())
    return grads, gcn(x, edge_index)

In [115]:
#vanilla GCN layer in pytorch without torch_geometric
class GraphConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GraphConvLayer, self).__init__()
        self.linear = nn.Linear(in_channels, out_channels) # provides the weight matrix for weighted sum of the neighbours' features

    def forward(self, x, adj_mat):
        # x --> N, in_channels
        # adj_mat --> N, N
        adj_hat = adj_mat + torch.eye(adj_mat.shape) # N, N
        d_hat = torch.diag(torch.pow(torch.sum(adj_mat, dim=1), -0.5)) # N, N
        a_hat = d_hat @ adj_mat @ d_hat # symmetrically normalized adjacency matrix
        # a_hat --> N, N
        x = self.linear(x)
        x = a_hat @ x
        return x # aggregation through summing the neighbours' features 


In [117]:
class GraphConvNet(nn.Module):
    def __init__(self, in_channels, hidden_dim, out_channels, num_layers):
        super(GraphConvNet, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(GraphConvLayer(in_channels, hidden_dim),
                           nn.LeakyReLU()
                        )
        for _ in range(num_layers - 2):
            self.layers.append(GraphConvLayer(hidden_dim, hidden_dim),
                              nn.LeakyReLU()
                            )
        self.layers.append(GraphConvLayer(hidden_dim, out_channels),
                          nn.LeakyReLU()
                          )
        self.model = nn.Sequential(*self.layers)
    
    def forward(self, x, adj_mat):
        return self.model(x, adj_mat)
